Jupyter notebook used for doing exploratory data analysis and data prep on the raw Oracle Advertising data created by the separate "Personal Data Datasets - camelot_pdf_reads.ipynb" notebook

In [1]:
import os
import numpy as np
import pandas as pd

In [18]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 300)

In [2]:
pd.set_option('display.max_colwidth', 150)

In [ ]:
# Get Oracle data from the Excel sheet that I copied the data into after extracting it from the pdf using Camelot
df = pd.read_excel('path to Oracle data in xlsx format')

In [ ]:
# visual inspection of the df showed that the data contained a set of records with 'Generated: Mon Oct 09 2023' in
# them and another set with 'Segment Detail View' in them
# these records separate broad categories of Oracle's inference data about me and do not add any useful information
# to the topic analysis
# so, I remove those records here

In [3]:
df = df.loc[~df['Segment Detail View'].str.contains('Generated: Mon Oct 09 2023')]
df

,Segment Detail View
1,A/B Test Groups > Group 08
2,Audiences by Oracle
3,"Audiences by Oracle > Auto, Cars and Trucks"
4,"Audiences by Oracle > Auto, Cars and Trucks > ..."
5,"Audiences by Oracle > Auto, Cars and Trucks > ..."
...,...
8266,Oracle DLX (Datalogix) > DLX Retail > Proximit...
8267,Oracle DLX (Datalogix) > DLX Subscription Serv...
8268,Oracle DLX (Datalogix) > DLX Subscription Serv...
8269,Oracle DLX (Datalogix) > DLX Telecommunication...


In [4]:
df = df.loc[~df['Segment Detail View'].str.contains('Segment Detail View')]
df

,Segment Detail View
1,A/B Test Groups > Group 08
2,Audiences by Oracle
3,"Audiences by Oracle > Auto, Cars and Trucks"
4,"Audiences by Oracle > Auto, Cars and Trucks > ..."
5,"Audiences by Oracle > Auto, Cars and Trucks > ..."
...,...
8266,Oracle DLX (Datalogix) > DLX Retail > Proximit...
8267,Oracle DLX (Datalogix) > DLX Subscription Serv...
8268,Oracle DLX (Datalogix) > DLX Subscription Serv...
8269,Oracle DLX (Datalogix) > DLX Telecommunication...


In [ ]:
# as you can see above, Oracle decsribes each inference in a hierarchial manner, with " > " separating
# the levels of hierarchy
# to make the visual EDA analysis a bit easier, I create a list of the hierarchies for each inference
df['Segment Detail as List'] = df['Segment Detail View'].apply(lambda x: x.split(' > '))

In [ ]:
# and I create a list of the hierarchies in reverse order
df['Segment Detail as List Reversed'] = df['Segment Detail View'].apply(lambda x: list(reversed(x.split(' > '))))

In [7]:
df

,Segment Detail View,Segment Detail as List,Segment Detail as List Reversed
1,A/B Test Groups > Group 08,"[A/B Test Groups, Group 08]","[Group 08, A/B Test Groups]"
2,Audiences by Oracle,[Audiences by Oracle],[Audiences by Oracle]
3,"Audiences by Oracle > Auto, Cars and Trucks","[Audiences by Oracle, Auto, Cars and Trucks]","[Auto, Cars and Trucks, Audiences by Oracle]"
4,"Audiences by Oracle > Auto, Cars and Trucks > ...","[Audiences by Oracle, Auto, Cars and Trucks, I...","[In-Market, Auto, Cars and Trucks, Audiences b..."
5,"Audiences by Oracle > Auto, Cars and Trucks > ...","[Audiences by Oracle, Auto, Cars and Trucks, I...","[Cars, Body Styles, In-Market, Auto, Cars and ..."
...,...,...,...
8266,Oracle DLX (Datalogix) > DLX Retail > Proximit...,"[Oracle DLX (Datalogix), DLX Retail, Proximity...","[Weight Watchers, Nutritional Supplements, Ret..."
8267,Oracle DLX (Datalogix) > DLX Subscription Serv...,"[Oracle DLX (Datalogix), DLX Subscription Serv...","[DLX Subscription Services, Oracle DLX (Datalo..."
8268,Oracle DLX (Datalogix) > DLX Subscription Serv...,"[Oracle DLX (Datalogix), DLX Subscription Serv...","[Mortgage, DLX Subscription Services, Oracle D..."
8269,Oracle DLX (Datalogix) > DLX Telecommunication...,"[Oracle DLX (Datalogix), DLX Telecommunication...","[DLX Telecommunications (Telco), Oracle DLX (D..."


In [7]:
df.reset_index(drop=True, inplace=True)

In [8]:
# here I create a new df based on the hierardhy lists in the prior df
# the new df has a separate column for each level in the hierarchy, up to 17 levels (the most levels found in the features)
df_new = pd.DataFrame(df['Segment Detail as List'].to_list())

In [9]:
df_new

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,A/B Test Groups,Group 08,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Audiences by Oracle,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Audiences by Oracle,"Auto, Cars and Trucks",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,Audiences by Oracle,"Auto, Cars and Trucks",In-Market,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,Audiences by Oracle,"Auto, Cars and Trucks",In-Market,Body Styles,Cars,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7793,Oracle DLX (Datalogix),DLX Retail,Proximity (Locations),Retailers,Nutritional Supplements,Weight Watchers,None,None,None,None,None,None,None,None,None,None,None
7794,Oracle DLX (Datalogix),DLX Subscription Services,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7795,Oracle DLX (Datalogix),DLX Subscription Services,Mortgage,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7796,Oracle DLX (Datalogix),DLX Telecommunications (Telco),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [13]:
df = df.join(df_new)

In [32]:
# here I try to get a sense for how descriptive the higher levels of hierarchy data are to see if we can use them for topic
# analysis rather than all levels
df[['Segment Detail View', 0, 1, 2]].groupby([0, 1, 2]).count()

Segment Detail View
0                      1                              2                                                            
Audiences by Oracle    Auto, Cars and Trucks          In-Market                                                  71
                                                      Interest (Affinity)                                        53
                                                      Parts, Service and Tires                                   12
                                                      Proximity (Locations)                                       5
                                                      Vehicle Owners                                             23
...                                                                                                             ...
Oracle DLX (Datalogix) DLX Finance                    Real Estate                                                 3
                       DLX Lifestyles                 Seniors                                                     1
                       DLX Retail                     Proximity (Locations)                                      48
                       DLX Subscription Services      Mortgage                                                    1
                       DLX Telecommunications (Telco) Mobile Carriers (Wireless Phone Service)                    1

[475 rows x 1 columns]

In [35]:
# pivot tables proved to be the best and easiest way to look at various combinations of the higher-level hierarchies
# to determine whcih ones to use in the topic analysis
#
# it was easier to move the data to an Excel sheet and then examine the various pivot table combinations through Excel
df.to_excel('Oracle/all_recs.xlsx')

In [70]:
# a combination of levels 2 and 3 seemed to be the best option for capturing meaning from the inrerences without
# including too many, often repetitive, hierarchy levels
df_levels_2_3 = pd.DataFrame()

In [71]:
df_levels_2_3['level_2'] = df[2]

In [72]:
df_levels_2_3['level_3'] = df[3]

In [73]:
df_levels_2_3 = df_levels_2_3.fillna(' ')

In [74]:
df_levels_2_3

,level_2,level_3
0,,
1,,
2,,
3,In-Market,
4,In-Market,Body Styles
...,...,...
7793,Proximity (Locations),Retailers
7794,,
7795,Mortgage,
7796,,


In [75]:
# combine the level 2 and level 3 data into a single column
df_levels_2_3['levels_2_and_3'] = df_levels_2_3['level_2'].str.cat(df_levels_2_3['level_3'], sep=' ')

In [76]:
df_levels_2_3

,level_2,level_3,levels_2_and_3
0,,,
1,,,
2,,,
3,In-Market,,In-Market
4,In-Market,Body Styles,In-Market Body Styles
...,...,...,...
7793,Proximity (Locations),Retailers,Proximity (Locations) Retailers
7794,,,
7795,Mortgage,,Mortgage
7796,,,


In [83]:
# drop empty records, caused by inferences that only had level 0 or 1 hierarchy data
df_levels_2_3 = df_levels_2_3.loc[~(df_levels_2_3['levels_2_and_3'] == '   ')]

In [84]:
df_levels_2_3

,level_2,level_3,levels_2_and_3
3,In-Market,,In-Market
4,In-Market,Body Styles,In-Market Body Styles
5,In-Market,Body Styles,In-Market Body Styles
6,In-Market,Body Styles,In-Market Body Styles
7,In-Market,Body Styles,In-Market Body Styles
...,...,...,...
7791,Proximity (Locations),Retailers,Proximity (Locations) Retailers
7792,Proximity (Locations),Retailers,Proximity (Locations) Retailers
7793,Proximity (Locations),Retailers,Proximity (Locations) Retailers
7795,Mortgage,,Mortgage


In [85]:
# use as input to BERTopic
df_levels_2_3.to_pickle('Oracle/levels_2_and_3.pkl')